In [1]:
import os
import sys
import re
from copy import deepcopy
from typing import List, Dict, Tuple
from glob import glob
import json
import yaml

from tqdm.auto import tqdm
import cv2

sys.path.append(r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\modules") # add path to scan customized module
from fileop import create_new_dir
import plt_show

# print("="*100, "\n")

In [2]:
def sortFishNameForDataset(fish_name_string:str) -> Tuple[int, str, int]:
    
    fish_name_for_dataset = fish_name_string.split(os.sep)[-1].split(".")[0]
    name_split_list = re.split(" |_|-", fish_name_for_dataset) # example_list : ['L', 'fish', '111', 'A', 'selected', '0']
    
    return int(name_split_list[2]), name_split_list[3], int(name_split_list[5])

Args

In [3]:
column = 5

load_dir_root = r"C:\Users\confocal_microscope\Desktop\{Test_by_fish}_Model_history"
model_name = "vit_b_16"
model_history = r"20230403_03_01_49_{Tested}_{84_epochs_AugOnFly}_{best}_{avg_f1_0.89034}"

Load `{yaml}_conifg`

In [4]:
load_dir = os.path.join(load_dir_root, model_name, model_history)

config_path = os.path.join(load_dir, r"train_config.yaml")
with open(config_path, 'r') as f_reader: 
    config = yaml.load(f_reader, Loader=yaml.FullLoader)

Set `vars`

In [5]:
# dataset
ap_dataset_root = os.path.normpath(config["dataset"]["root"])
dataset_name = config["dataset"]["name"]
dataset_gen_method = config["dataset"]["gen_method"]
dataset_param_name = config["dataset"]["para_mname"]

Generate `path_vars`

In [6]:
# dataset
dataset_dir = os.path.join(ap_dataset_root, dataset_name, dataset_gen_method, dataset_param_name)
test_selected_dir = os.path.join(dataset_dir, "test", "selected")
test_drop_dir = os.path.join(dataset_dir, "test", "drop")

# cam, cam_gallery
cam_result_root = os.path.join(load_dir, "GradCAM_result_with_text")
cam_result_gallery = os.path.join(load_dir, "!---Gallery")

# cam_gallery_class_dir
logs_path = os.path.join(dataset_dir, r"{Logs}_train_selected_summary.log")
with open(logs_path, 'r') as f_reader: class_counts: Dict[str, int] = json.load(f_reader)
for key, _ in class_counts.items():
    gallery_class_dir = os.path.join(cam_result_gallery, key)
    create_new_dir(gallery_class_dir, display_in_CLI=False)

Run

In [7]:
fish_name_for_dataset_list = [ path.split(os.sep)[-1] for path in glob(os.path.normpath(f"{cam_result_root}/*")) ]
pbar_n_img = tqdm(total=len(fish_name_for_dataset_list), desc="Gallery ")


for fish_name_for_dataset in fish_name_for_dataset_list:
    
    
    pbar_n_img.desc = f"Generate ' {fish_name_for_dataset} ' "
    pbar_n_img.refresh()
    
    
    fish_name_for_dataset_split_list = re.split(" |_|-", fish_name_for_dataset)
    
    test_selected_path_list = glob(os.path.normpath((f"{test_selected_dir}/{fish_name_for_dataset_split_list[0]}/"
                                                     f"{fish_name_for_dataset}_selected_*.tiff")))
    
    test_drop_path_list = glob(os.path.normpath((f"{test_drop_dir}/{fish_name_for_dataset_split_list[0]}/"
                                                 f"{fish_name_for_dataset}_drop_*.tiff")))

    cam_result_path_list = glob(os.path.normpath(f"{cam_result_root}/{fish_name_for_dataset}/*.tiff"))

    orig_path_list = deepcopy(test_selected_path_list)
    orig_path_list.extend(test_drop_path_list)
    orig_path_list.sort(key=sortFishNameForDataset)
    orig_img_list = [ cv2.imread(img_path) for img_path in orig_path_list ]
    # plot with 'Auto Row Calculation'
    kwargs_plot_with_imglist_auto_row = {
        "img_list"   : orig_img_list,
        "column"     : column,
        "fig_title"  : f"( original ) {fish_name_for_dataset} : {orig_img_list[-1].shape[:2]}",
        "save_path"  : f"{cam_result_gallery}/{fish_name_for_dataset_split_list[0]}/{fish_name_for_dataset}_orig.png",
        "show_fig"   : False
    }
    plt_show.plot_with_imglist_auto_row(**kwargs_plot_with_imglist_auto_row)
    
    
    cam_overlap_path_list = deepcopy(cam_result_path_list)
    cam_overlap_path_list.extend(test_drop_path_list)
    cam_overlap_path_list.sort(key=sortFishNameForDataset)
    cam_overlap_img_list = [ cv2.imread(img_path) for img_path in cam_overlap_path_list ]
    # plot with 'Auto Row Calculation'
    kwargs_plot_with_imglist_auto_row = {
        "img_list"   : cam_overlap_img_list,
        "column"     : column,
        "fig_title"  : f"( cam overlap ) {fish_name_for_dataset} : {cam_overlap_img_list[-1].shape[:2]}",
        "save_path"  : f"{cam_result_gallery}/{fish_name_for_dataset_split_list[0]}/{fish_name_for_dataset}_overlap.png",
        "show_fig"   : False
    }
    plt_show.plot_with_imglist_auto_row(**kwargs_plot_with_imglist_auto_row)
    
    
    pbar_n_img.update(1)
    pbar_n_img.refresh()


pbar_n_img.close()

Gallery :   0%|          | 0/396 [00:00<?, ?it/s]